In [1]:
import os
import torch
import torch.utils.data
from torch import nn, optim
from torch.autograd import Variable
from torch.nn import functional as F
from torchvision import datasets, transforms

from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader

import numpy as np
from time import time
import copy

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

CUDA = torch.cuda.is_available()

In [2]:
RAW_DATA_DIR = "../Data/Raw/"
PROCESSED_DATA_DIR = "../Data/Processed/Regression/"

class CovariateDataset(Dataset):
    def __init__(self, file_name_pattern, file_name_args, train_size=0.8, test_size=0.2, test_train_complement=True):
        self.train = True
        self.test_on_all = False
        
        self.file_name = file_name_pattern.format(*file_name_args, "covar")
        self.assignment_file_name = file_name_pattern.format(*file_name_args, "assignment")
        
        self.data = np.loadtxt(RAW_DATA_DIR + self.file_name + ".csv", delimiter=",")[:, 1:] # remove bias
        self.assignment_data = np.loadtxt(
            RAW_DATA_DIR + self.assignment_file_name + ".csv", delimiter=",").astype(int)
        
        self.all_indeces = np.array(range(len(self.data)))
        treat_indeces = self.all_indeces[self.assignment_data.astype(int) == 1]
        control_indeces = self.all_indeces[self.assignment_data.astype(int) == 0]
        
        num_training = int(len(self.data)*train_size)
        
        self.train_indeces = np.random.choice(self.all_indeces, num_training, replace=False)
        if test_train_complement:
            self.test_indeces = list(set(self.all_indeces)^set(self.train_indeces))      
        else:
            self.test_indeces = np.random.choice(self.all_indeces, int(len(self.data)*(1-test_size)), replace=False)
        
        num_treated_in_train = len(np.intersect1d(treat_indeces, self.train_indeces, assume_unique=True))
        num_control_in_train = num_training - num_treated_in_train
        
        treat_weight = num_training / (2 * num_treated_in_train)
        control_weight = num_training / (2 * num_control_in_train)
        
        weighter = np.vectorize(lambda index: treat_weight if index in\
            treat_indeces else control_weight)
        
        self.weights = weighter(self.all_indeces)
        
    def active_data(self, index=0):
        if self.train:
            return self.data[self.train_indeces], self.assignment_data[self.train_indeces], \
                self.weights[self.train_indeces][index]
        else:
            if self.test_on_all:
                indeces = self.all_indeces
            else: 
                indeces = self.test_indeces
            
            return self.data[indeces], self.assignment_data[indeces], 1
            
    def __getitem__(self, index):
        covar_data, assignment_data, weight_data = self.active_data(index)
        class_vector = np.zeros(2)
        class_vector[int(assignment_data[index])] = 1
        
        return (covar_data[index], class_vector, weight_data)

    def __len__(self):
        return self.active_data()[0].shape[0]
    
    def save_processed_data(self, data):
        name = PROCESSED_DATA_DIR + self.file_name+".csv"
        np.savetxt(name, data, delimiter=",")
        
def get_datasets(file_name_format, file_name_args, **kwargs):
    train_set = CovariateDataset(file_name_format, file_name_args, **kwargs)
    test_set = copy.deepcopy(train_set)
    test_set.train = False

    predict_set = copy.deepcopy(train_set)
    predict_set.train = False
    predict_set.test_on_all = True
    
    return train_set, test_set, predict_set

In [3]:
# Based on an example from https://github.com/pytorch/examples/blob/master/vae/main.py
# Extended to place a different prior on binary vs normal vars

class Regressor(nn.Module):
    def __init__(self):
        super(Regressor, self).__init__()
        
        INTERMEDIATE_DIMS_1 = 16
        INTERMEDIATE_DIMS_2 = 16
        INTERMEDIATE_DIMS_3 = 16
        INTERMEDIATE_DIMS_4 = 16
#         INTERMEDIATE_DIMS_5 = 16
#         INTERMEDIATE_DIMS_6 = 8

        FEATURES = 10

        LOSS_SCALE = 1

        # ENCODER LAYERS
        self.dense1 = nn.Linear(FEATURES, INTERMEDIATE_DIMS_1)
        self.dense2 = nn.Linear(INTERMEDIATE_DIMS_1, INTERMEDIATE_DIMS_2)
        self.dense3 = nn.Linear(INTERMEDIATE_DIMS_2, INTERMEDIATE_DIMS_3)
        self.dense4 = nn.Linear(INTERMEDIATE_DIMS_3, INTERMEDIATE_DIMS_4)
#         self.dense5 = nn.Linear(INTERMEDIATE_DIMS_4, INTERMEDIATE_DIMS_5)
#         self.dense6 = nn.Linear(INTERMEDIATE_DIMS_5, INTERMEDIATE_DIMS_6)
        self.dense5 = nn.Linear(INTERMEDIATE_DIMS_4, 2)
        
        # Activations
        self.softmax = nn.Softmax(dim=1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.6)

    def forward(self, x):
        h1 = self.dropout(self.relu(self.dense1(x)))
        h2 = self.dropout(self.relu(self.dense2(h1)))
        h3 = self.dropout(self.relu(self.dense3(h2)))
        h4 = self.dropout(self.relu(self.dense4(h3)))
#         h5 = self.dropout(self.relu(self.dense5(h4)))
#         h6 = self.dropout(self.relu(self.dense6(h5)))
        
        return self.softmax(self.dense5(h4))



def train(model, optimizer, epoch, train_loader, log_results=False):
    model.train()
    train_loss = 0

    for batch_idx, (data, target_class, weights) in enumerate(train_loader):
        data = Variable(data)
        target_class = Variable(target_class)
        weights = Variable(weights)
        
        data = data.float()
        target_class = target_class.float()
        weights = weights.float()
        
        if CUDA:
            data = data.cuda()
            target_class = target_class.cuda()
            weights = weights.cuda()
        
        optimizer.zero_grad()

        output_propensity = model(data)
        
        # calculate loss
        loss_criterion = nn.BCELoss(weight=weights.view(weights.shape[0], 1), size_average=False)
        loss = loss_criterion(output_propensity, target_class)

        train_loss += loss.data[0]
        
        # Find the gradient and descend
        loss.backward()
        optimizer.step()
        
    if log_results:
        print('====> Epoch: {} Average loss: {:.8f}'.format(
              epoch, train_loss / len(train_loader.dataset)))
        
        
def test(model, epoch, test_loader):
    # toggle model to test / inference mode
    model.eval()
    test_loss = 0

    for i, (data, target_class, weights) in enumerate(test_loader):
        data = Variable(data, volatile=True)
        target_class = Variable(target_class, volatile=True)
        weights = Variable(weights, volatile=True)
        
        data = data.float()
        target_class = target_class.float()
        weights = weights.float()
        
        if CUDA:
            data = data.cuda()
            target_class = target_class.cuda()
            weights = weights.cuda()

        output_propensity = model(data)
        
        # calculate loss
        loss_criterion = nn.BCELoss(weight=weights.view(weights.shape[0], 1), size_average=False)
        loss = loss_criterion(output_propensity, target_class)
        test_loss += loss.data[0]

    test_loss /= len(test_loader.dataset)
    
    if CUDA:
        output_propensity = output_propensity.cpu()
        target_class = target_class.cpu()
        
    score = accuracy(output_propensity.data.numpy(), target_class.data.numpy(), verbose=False)
    print('====> Test set loss: {:.4f}, {}%'.format(test_loss, score*100))
    
def predict(model, predict_loader):
    # Show reconstruction
    model.eval()
    print("Training state: ", model.training)
    
    original_data, targets, _ = next(iter(predict_loader))
    
    original_data = Variable(original_data)
    original_data = original_data.float()
    
    if CUDA:
        original_data = original_data.cuda()
        
    return original_data, targets, model(original_data)

def accuracy(output_data, targets, verbose=True):
        
    classes = np.argmax(output_data, axis=1)
    targets = np.argmax(targets, axis=1)
    
    if verbose:
        print(classification_report(targets, classes))
    return accuracy_score(targets, classes)

In [4]:
def train_model(model_class, train_set, test_set, predict_set, dataset_number, verbose=True, model=None):
    if model is None:
        model = model_class()
        if CUDA:
            model = model.cuda()

    num_epochs = 750
    train_batch_size = 64
    test_batch_size = 250
    learning_rate = 1e-3
    lr_sched = True
         
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, [int(num_epochs/5), int(num_epochs/2)], gamma=0.1)

    train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True)
    test_loader = DataLoader(test_set, batch_size=test_batch_size, shuffle=True)
    predict_loader = DataLoader(predict_set, batch_size=1000, shuffle=False)
    
    for epoch in range(1, num_epochs+1):
        checkpoint_interval = int(num_epochs/10)
        
        if lr_sched:
            scheduler.step()

        log = False
        if epoch%checkpoint_interval == 0:
            log = True
            
        train(model, optimizer, epoch, train_loader, log_results=log)
        if log:
            test(model, epoch, test_loader)
    
    original_data, targets, output = predict(model, predict_loader)
    if CUDA:
        output = output.cpu()
        targets = targets.cpu()
    
    return model, original_data, targets, output

def encode_data(dataset, output_data):
    
    if CUDA:
        output_data = output_data.cpu()
        
    dataset.save_processed_data(output_data.data.numpy()[:, 1])


In [7]:
def run_logistic(train_set, verbose=True):
    model = LogisticRegression(class_weight="balanced")
    
    X = train_set.data
    y = train_set.assignment_data

    X_train = X[train_set.train_indeces]
    X_test = X[train_set.test_indeces]
    y_train = y[train_set.train_indeces]
    y_test = y[train_set.test_indeces]

    model.fit(X_train, y_train)
    predictions = model.predict(X)
    
    if verbose:
        print(classification_report(y, predictions))
    
    return accuracy_score(y, predictions)

In [5]:
train_set, test_set, predict_set = get_datasets(
    "n_{}_model_{}_v_{}_{}_data", [1000, "G_mod_nadd_mod_nlin", 1],
    train_size=0.8, test_train_complement=True)

start = time()
trained_model, original_data, targets, output = \
    train_model(Regressor, train_set, test_set,predict_set, 1,verbose=True)
print("Elapsed: ", time() - start)


acc = accuracy(output.data.cpu().numpy(), targets.numpy(), verbose=False)
print("Complete set accuracy: {}%".format(acc*100))

encode_data(train_set, output)

====> Epoch: 75 Average loss: 1.28923667
====> Test set loss: 1.2808, 63.5%
====> Epoch: 150 Average loss: 1.23414843
====> Test set loss: 1.2731, 60.5%
====> Epoch: 225 Average loss: 1.22775954
====> Test set loss: 1.2520, 63.0%
====> Epoch: 300 Average loss: 1.22451996
====> Test set loss: 1.2490, 63.0%
====> Epoch: 375 Average loss: 1.21432436
====> Test set loss: 1.2540, 62.0%
====> Epoch: 450 Average loss: 1.26265797
====> Test set loss: 1.2523, 62.0%
====> Epoch: 525 Average loss: 1.22552528
====> Test set loss: 1.2508, 62.0%
====> Epoch: 600 Average loss: 1.27635715
====> Test set loss: 1.2503, 62.0%
====> Epoch: 675 Average loss: 1.22551006
====> Test set loss: 1.2490, 62.5%
====> Epoch: 750 Average loss: 1.21764433
====> Test set loss: 1.2480, 62.5%
Training state:  False
Elapsed:  28.841941833496094
Complete set accuracy: 64.3%


### Runner

In [10]:
assignment_model_names = ['A_add_lin', 'B_add_mild_nlin', 'C_add_mod_nlin', 'D_mild_nadd_lin',
                     'E_mild_nadd_mild_nlin', 'F_mod_nadd_lin', 'G_mod_nadd_mod_nlin']

nn_accuracies = []
log_accuracies = []

for dataset_number in range(50, 100):
    print("Starting run for Dataset {}".format(dataset_number))
    
    for model_name in assignment_model_names:
        print("---- Running for model name: ", model_name)
        
        start = time()

        train_set, test_set, predict_set = get_datasets(
            "n_{}_model_{}_v_{}_{}_data", [1000, model_name, dataset_number], train_size=0.8)

        trained_model, original_data, targets, output = \
            train_model(Regressor, train_set, test_set,predict_set, 1,verbose=True)
        
        nn_acc = accuracy(output.data.cpu().numpy(), targets.numpy(), verbose=False)
        print("Complete set accuracy: {}%".format(nn_acc*100))
        
        log_acc = run_logistic(train_set, verbose=False)
        print("Log accuracy: {}%".format(log_acc*100))
        
        nn_accuracies.append(nn_acc)
        log_accuracies.append(log_acc)

        encode_data(train_set, output)

        print("---- Done in ", time() - start, " seconds\n")
                
    print("================\n\n")

Starting run for Dataset 50
---- Running for model name:  A_add_lin
====> Epoch: 75 Average loss: 1.26398590
====> Test set loss: 1.2614, 66.0%
====> Epoch: 150 Average loss: 1.26268904
====> Test set loss: 1.2431, 67.0%
====> Epoch: 225 Average loss: 1.25635259
====> Test set loss: 1.2358, 68.0%
====> Epoch: 300 Average loss: 1.25354322
====> Test set loss: 1.2361, 68.0%
====> Epoch: 375 Average loss: 1.26769718
====> Test set loss: 1.2370, 67.0%
====> Epoch: 450 Average loss: 1.24548035
====> Test set loss: 1.2372, 67.0%
====> Epoch: 525 Average loss: 1.26685104
====> Test set loss: 1.2355, 67.0%
====> Epoch: 600 Average loss: 1.27289847
====> Test set loss: 1.2351, 67.0%
====> Epoch: 675 Average loss: 1.25684374
====> Test set loss: 1.2350, 67.0%
====> Epoch: 750 Average loss: 1.24228590
====> Test set loss: 1.2344, 67.0%
Training state:  False
Complete set accuracy: 68.2%
Log accuracy: 69.69999999999999%
---- Done in  48.5192129611969  seconds

---- Running for model name:  B_add_m

====> Epoch: 750 Average loss: 1.18895810
====> Test set loss: 1.1294, 74.5%
Training state:  False
Complete set accuracy: 74.4%
Log accuracy: 70.1%
---- Done in  59.284404039382935  seconds

---- Running for model name:  C_add_mod_nlin
====> Epoch: 75 Average loss: 1.34308517
====> Test set loss: 1.3029, 61.0%
====> Epoch: 150 Average loss: 1.21152949
====> Test set loss: 1.2306, 68.5%
====> Epoch: 225 Average loss: 1.22344339
====> Test set loss: 1.2274, 70.0%
====> Epoch: 300 Average loss: 1.22765281
====> Test set loss: 1.2190, 70.0%
====> Epoch: 375 Average loss: 1.27256130
====> Test set loss: 1.2216, 69.0%
====> Epoch: 450 Average loss: 1.24415952
====> Test set loss: 1.2212, 69.0%
====> Epoch: 525 Average loss: 1.24285086
====> Test set loss: 1.2205, 69.5%
====> Epoch: 600 Average loss: 1.23563943
====> Test set loss: 1.2205, 69.5%
====> Epoch: 675 Average loss: 1.24380835
====> Test set loss: 1.2203, 69.5%
====> Epoch: 750 Average loss: 1.23293856
====> Test set loss: 1.2198, 

====> Epoch: 525 Average loss: 1.20558383
====> Test set loss: 1.0662, 76.5%
====> Epoch: 600 Average loss: 1.18037009
====> Test set loss: 1.0661, 76.5%
====> Epoch: 675 Average loss: 1.19549920
====> Test set loss: 1.0660, 76.0%
====> Epoch: 750 Average loss: 1.15191770
====> Test set loss: 1.0661, 76.0%
Training state:  False
Complete set accuracy: 76.8%
Log accuracy: 74.3%
---- Done in  52.599098682403564  seconds

---- Running for model name:  E_mild_nadd_mild_nlin
====> Epoch: 75 Average loss: 1.28113245
====> Test set loss: 1.1641, 73.0%
====> Epoch: 150 Average loss: 1.23681130
====> Test set loss: 1.1242, 73.0%
====> Epoch: 225 Average loss: 1.25647645
====> Test set loss: 1.1238, 73.5%
====> Epoch: 300 Average loss: 1.23190334
====> Test set loss: 1.1210, 73.5%
====> Epoch: 375 Average loss: 1.20395102
====> Test set loss: 1.1196, 73.0%
====> Epoch: 450 Average loss: 1.21775053
====> Test set loss: 1.1193, 73.0%
====> Epoch: 525 Average loss: 1.21032036
====> Test set loss: 1

====> Epoch: 300 Average loss: 1.16370184
====> Test set loss: 1.1951, 70.0%
====> Epoch: 375 Average loss: 1.18487591
====> Test set loss: 1.1958, 69.5%
====> Epoch: 450 Average loss: 1.18255072
====> Test set loss: 1.1947, 69.5%
====> Epoch: 525 Average loss: 1.18799205
====> Test set loss: 1.1942, 69.5%
====> Epoch: 600 Average loss: 1.18753402
====> Test set loss: 1.1931, 70.0%
====> Epoch: 675 Average loss: 1.18654081
====> Test set loss: 1.1936, 69.5%
====> Epoch: 750 Average loss: 1.20012967
====> Test set loss: 1.1938, 69.5%
Training state:  False
Complete set accuracy: 74.7%
Log accuracy: 71.2%
---- Done in  52.260791063308716  seconds

---- Running for model name:  G_mod_nadd_mod_nlin
====> Epoch: 75 Average loss: 1.31008127
====> Test set loss: 1.2631, 72.5%
====> Epoch: 150 Average loss: 1.21864770
====> Test set loss: 1.1611, 70.5%
====> Epoch: 225 Average loss: 1.25602608
====> Test set loss: 1.1617, 70.0%
====> Epoch: 300 Average loss: 1.20970485
====> Test set loss: 1.1

====> Epoch: 75 Average loss: 1.26119559
====> Test set loss: 1.1970, 72.0%
====> Epoch: 150 Average loss: 1.22385476
====> Test set loss: 1.1287, 76.5%
====> Epoch: 225 Average loss: 1.21674015
====> Test set loss: 1.1244, 76.5%
====> Epoch: 300 Average loss: 1.21466518
====> Test set loss: 1.1257, 76.5%
====> Epoch: 375 Average loss: 1.22715649
====> Test set loss: 1.1233, 76.0%
====> Epoch: 450 Average loss: 1.19884297
====> Test set loss: 1.1238, 76.0%
====> Epoch: 525 Average loss: 1.19015323
====> Test set loss: 1.1236, 76.0%
====> Epoch: 600 Average loss: 1.21035187
====> Test set loss: 1.1231, 76.0%
====> Epoch: 675 Average loss: 1.21429519
====> Test set loss: 1.1219, 76.0%
====> Epoch: 750 Average loss: 1.20268043
====> Test set loss: 1.1221, 76.0%
Training state:  False
Complete set accuracy: 75.5%
Log accuracy: 72.7%
---- Done in  51.72238206863403  seconds

---- Running for model name:  B_add_mild_nlin
====> Epoch: 75 Average loss: 1.27125936
====> Test set loss: 1.2499, 7

...

====> Epoch: 750 Average loss: 1.15219206
====> Test set loss: 1.0872, 73.0%
Training state:  False
Complete set accuracy: 77.7%
Log accuracy: 73.7%
---- Done in  63.45147895812988  seconds

---- Running for model name:  G_mod_nadd_mod_nlin
====> Epoch: 75 Average loss: 1.33929375
====> Test set loss: 1.3080, 67.5%
====> Epoch: 150 Average loss: 1.22446687
====> Test set loss: 1.1672, 69.5%
====> Epoch: 225 Average loss: 1.25642081
====> Test set loss: 1.1678, 69.0%
====> Epoch: 300 Average loss: 1.25305199
====> Test set loss: 1.1578, 70.5%
====> Epoch: 375 Average loss: 1.18723804
====> Test set loss: 1.1549, 71.5%
====> Epoch: 450 Average loss: 1.18063313
====> Test set loss: 1.1549, 71.5%
====> Epoch: 525 Average loss: 1.23243188
====> Test set loss: 1.1544, 71.5%
====> Epoch: 600 Average loss: 1.19984212
====> Test set loss: 1.1527, 71.5%
====> Epoch: 675 Average loss: 1.19981278
====> Test set loss: 1.1515, 71.5%
====> Epoch: 750 Average loss: 1.26666116
====> Test set loss: 1.14

====> Epoch: 450 Average loss: 1.23303181
====> Test set loss: 1.1643, 71.0%
====> Epoch: 525 Average loss: 1.18501314
====> Test set loss: 1.1631, 71.0%
====> Epoch: 600 Average loss: 1.19011609
====> Test set loss: 1.1631, 71.0%
====> Epoch: 675 Average loss: 1.16975159
====> Test set loss: 1.1625, 71.0%
====> Epoch: 750 Average loss: 1.17140144
====> Test set loss: 1.1618, 71.0%
Training state:  False
Complete set accuracy: 73.5%
Log accuracy: 71.3%
---- Done in  61.754287242889404  seconds

---- Running for model name:  B_add_mild_nlin
====> Epoch: 75 Average loss: 1.28859195
====> Test set loss: 1.2388, 66.5%
====> Epoch: 150 Average loss: 1.21793306
====> Test set loss: 1.2159, 66.5%
====> Epoch: 225 Average loss: 1.22783456
====> Test set loss: 1.2074, 65.5%
====> Epoch: 300 Average loss: 1.14515783
====> Test set loss: 1.2088, 67.0%
====> Epoch: 375 Average loss: 1.20549406
====> Test set loss: 1.2103, 67.0%
====> Epoch: 450 Average loss: 1.20922149
====> Test set loss: 1.2103,

====> Epoch: 225 Average loss: 1.21528973
====> Test set loss: 1.1484, 73.0%
====> Epoch: 300 Average loss: 1.19883053
====> Test set loss: 1.1502, 73.5%
====> Epoch: 375 Average loss: 1.19372270
====> Test set loss: 1.1433, 73.5%
====> Epoch: 450 Average loss: 1.22063066
====> Test set loss: 1.1434, 73.5%
====> Epoch: 525 Average loss: 1.17811407
====> Test set loss: 1.1437, 73.5%
====> Epoch: 600 Average loss: 1.19477195
====> Test set loss: 1.1442, 73.5%
====> Epoch: 675 Average loss: 1.16150311
====> Test set loss: 1.1425, 73.5%
====> Epoch: 750 Average loss: 1.21931358
====> Test set loss: 1.1419, 73.5%
Training state:  False
Complete set accuracy: 75.6%
Log accuracy: 69.39999999999999%
---- Done in  62.7699658870697  seconds

---- Running for model name:  D_mild_nadd_lin
====> Epoch: 75 Average loss: 1.25536298
====> Test set loss: 1.2372, 69.0%
====> Epoch: 150 Average loss: 1.17252641
====> Test set loss: 1.2144, 68.0%
====> Epoch: 225 Average loss: 1.16663722
====> Test set lo

====> Epoch: 75 Average loss: 1.19221242
====> Test set loss: 1.1841, 67.0%
====> Epoch: 150 Average loss: 1.09827701
====> Test set loss: 1.1607, 67.0%
====> Epoch: 225 Average loss: 1.07255318
====> Test set loss: 1.1548, 67.0%
====> Epoch: 300 Average loss: 1.17135214
====> Test set loss: 1.1541, 68.0%
====> Epoch: 375 Average loss: 1.13167843
====> Test set loss: 1.1529, 68.0%
====> Epoch: 450 Average loss: 1.11934855
====> Test set loss: 1.1528, 68.0%
====> Epoch: 525 Average loss: 1.15584826
====> Test set loss: 1.1526, 68.0%
====> Epoch: 600 Average loss: 1.11809550
====> Test set loss: 1.1530, 68.0%
====> Epoch: 675 Average loss: 1.14056297
====> Test set loss: 1.1525, 68.5%
====> Epoch: 750 Average loss: 1.14117324
====> Test set loss: 1.1521, 68.5%
Training state:  False
Complete set accuracy: 75.9%
Log accuracy: 73.6%
---- Done in  67.69600224494934  seconds

---- Running for model name:  F_mod_nadd_lin
====> Epoch: 75 Average loss: 1.25152396
====> Test set loss: 1.1154, 73

====> Epoch: 75 Average loss: 1.33847738
====> Test set loss: 1.3232, 70.0%
====> Epoch: 150 Average loss: 1.29895312
====> Test set loss: 1.2328, 73.5%
====> Epoch: 225 Average loss: 1.25225529
====> Test set loss: 1.2310, 72.0%
====> Epoch: 300 Average loss: 1.28749635
====> Test set loss: 1.2276, 71.5%
====> Epoch: 375 Average loss: 1.28837023
====> Test set loss: 1.2219, 71.0%
====> Epoch: 450 Average loss: 1.26001772
====> Test set loss: 1.2222, 71.0%
====> Epoch: 525 Average loss: 1.26106489
====> Test set loss: 1.2222, 71.0%
====> Epoch: 600 Average loss: 1.25701874
====> Test set loss: 1.2222, 71.0%
====> Epoch: 675 Average loss: 1.25123661
====> Test set loss: 1.2221, 71.0%
====> Epoch: 750 Average loss: 1.26453034
====> Test set loss: 1.2226, 71.0%
Training state:  False
Complete set accuracy: 74.0%
Log accuracy: 65.8%
---- Done in  65.58177280426025  seconds



Starting run for Dataset 99
---- Running for model name:  A_add_lin
====> Epoch: 75 Average loss: 1.28234618
====> 